# Scaling of Missing Persons across North America, Mexico, and South America:
## Missing Persons w/ Dynamical Systems Lab @ NYU
### Ayush Sarkar

# Introduction: 
* To better understand the parallels in the number of missing persons cases in urban areas, there was an interest in investigating the scaling rates between two different countries and comparing them. 
* We considered many different South American countries, including Mexico, Columbia, Brazil, and Argentina. Despite considering so many other countries, Mexico had the most complete and similar structure to the United States, although Colombia did seem promising.

# Methodology: 
* From investigation, it seems as if there are few public missing persons databases available due to the sensitive nature of these cases and the family members involved. There is seemingly a federal-level database, although it is not accessible to public under normal circumstances. Thus, we consider the National Missing and Unidentified Persons System, or NamUs for short, as which was scraped as of 07/17/2025 to generate the supplemental json file included. 
  * The NamUs database contained about 25,629 entries as of 07/17/2025.
* We consider three main datasets in regards to developing the United States CBSA case sums and population estimates for scaling: 
  (1) NamUs Missing Persons Dataset (as of 07/17/2025)
  (2) SEER Age-adjusted Population Estimates 1969-2022
  (3) QCEW-County-MSA-CSA Crosswalk (Contains historical crosswalks)

# Missing Persons & Population Data Sources

---

## United States Missing Persons & Population Data

### Census & Population Data
- **University of Brown’s Guide to Census Data**  
  https://libguides.brown.edu/census/histmicro

- **US Census Population Estimates by County (1969–2022)**  
  https://seer.cancer.gov/popdata.thru.2022/download.html

- **SEER FIPS Updates (1969–2022)**  
  https://seer.cancer.gov/seerstat/variables/countyattribs/time-dependent.html

- **David Dorn FIPS Updates (1980–2021)**  
  https://www.ddorn.net/data/FIPS_County_Code_Changes.pdf

- **NHGIS – Historical Shapefiles**  
  https://data2.nhgis.org/main

---

### Missing Persons Scrapers & Tools

#### NightOwlRecon (NamUS Data)
- **GitHub Repository**  
  https://github.com/NightOwlRecon/NamUs-Data/blob/main/namus.py

- **Extracted `.json` Dataset**  
  https://drive.google.com/file/d/1k8PRzRlwE_Ti52enW4qjNX0bInr5Pf0g/view?usp=sharing

#### Prepager (NamUS Scraper)
- **GitHub Repository**  
  https://github.com/Prepager/namus-scraper

---

### Crosswalks & Spatial References
- **Historical County / MSA / CSA Crosswalks (BLS)**  
  https://www.bls.gov/cew/classifications/areas/county-msa-csa-crosswalk.htm

- **Connecticut Town Crosswalks (2023–Present)**  
  https://data.ct.gov/Local-Government/Connecticut-Towns-Crosswalk-with-Tax-Codes-and-FIP/5hqs-h5c3/about_data

- **Connecticut FIPS Codes for Planning Regions (AP Elections API)**  
  https://developer.ap.org/ap-elections-api/docs/CT_FIPS_Codes_forPlanningRegions.htm

---

## South American & Latin American Missing Persons Data

### Mexico
- **INEGI Missing & Unaccounted-for Persons Dataset**  
  https://figshare.com/articles/dataset/Missing_and_Unaccounted-for_People_in_Mexico_1960s_2025_/28283000

- **Governmental Crime & Missing Persons Report**  
  https://www.gob.mx/sesnsp/acciones-y-programas/incidencia-delictiva-del-fuero-comun-nueva-metodologia

### New Mexico
- **INEGI Missing Persons (ELCRI)**  
  https://elcri.men/en/about/

### Colombia
- **SIEDCO Missing Persons Statistics (National Police)**  
  https://www.policia.gov.co/estadistica-delictiva

### Argentina
- **Personas Perdidas**  
  http://personasperdidas.org.ar/looking_for_their_families

- **Police Reports (2020–2024)**  
  https://www.datos.gob.ar/dataset/justicia-lucha-contra-trata-personas---llamados-linea-145---denuncias/archivo/justicia_4b786057-973f-4bd6-9594-8b74233ad9b1

- **Federal System for Searching for Missing and Lost Persons – Management Report**  
  https://www.argentina.gob.ar/sites/default/files/ministerio-seguridad-argentina-informe-gestion-sifebu-2024.pdf

### Brazil
- **Missing Persons & Forced Disappearances (Academic Article)**  
  https://www.sciencedirect.com/science/article/pii/S2665910722000330

---

## Anecdotal / Qualitative Data & Additional Resources

- **The Charley Project – Geographic Case Search (Mexico)**  
  https://charleyproject.org/case-searches/geographical-cases?region=Mexico

- **The Doe Network**  
  https://www.doenetwork.org/

- **The Lost People – NamUS Choropleth Mapping**  
  https://jseibel55.github.io/The-Lost-People/#collapseThree

- **A Consistent County-Level Spatial Crosswalk Since 1790**  
  https://fpeckert.me/papers/egp-spatialcrosswalk.pdf
